In [11]:
import pyvisa as visa#import pyvisa as pv #from pyvisa import *
from typing import List
import ctypes

In [12]:
from nidmm import *

In [13]:
from nimodinst._visatype import * #import nimodinst._visatype as _visatype

In [14]:
#флаг остановки программы
Stop_program = 0
#функция настройки приборов
# rmSession: ViSession
# scope: ViSession
# gen: ViSession #Объект сессии(общий, с осцилограффом, с генератором)
# resourceList: List[str]   #ViFindList  #Объект для записи всех найденных устройств
# numOSC: ViUInt32
# numGEN: ViUInt32 #Объект для записи номера устройства(осцилографа, генератора)
# status: ViStatus #Объект для записи статуса выполняемой команды
# readOSC:str#=[None]*64 #константа для записи ответа от осцилограффа
# readGEN:str#=[None]*64 #константа для записи ответа от генератора
# usbOSC:str #Константа для записи имени осцилограффа(пути до осцилографа?)
# usbGEN:str #Константа для записи имени генератора(пути до генератора?)

In [20]:
def SettingUpDevices():
    #Открываем сессию с менеджером ресурсов

    rm = visa.ResourceManager()
    rm.list_resources()
    #('ASRL1::INSTR', 'ASRL2::INSTR', 'GPIB0::14::INSTR')
    #my_instrument = rm.open_resource('GPIB0::14::INSTR')
    #print(my_instrument.query('*IDN?'))
    
    scope = rm.open_resource("?*USB?*INSTR{VI_ATTR_MANF_ID==0x1AB1 && VI_ATTR_MODEL_CODE==0x517}")
    scope.write("*rst; status:preset; *cls")#сброс настроек
    
    if (success > StatusCode):
        print("ERROR_OSC Oscilloscope not found\n")
        Stop_program = 1
    else: print("Oscilloscope - OK\n")

  #Ищем ЮСБ генератор(модель генератора указываем в аргументе)
    gen = rm.open_resource("?*USB?*INSTR{VI_ATTR_MANF_ID==0x1AB1 && VI_ATTR_MODEL_CODE==0x642}")
    gen.write("*rst; status:preset; *cls")#сброс настроек
    
    if (success > StatusCode):
        print("ERROR_GEN Generator not found\n")
        Stop_program = 1
    else: print("Generator - OK\n")

  #Открываем сессию с осцилографом
    #status = viOpen(rmSession, usbOSC, VI_NULL, VI_NULL, scope)
    if (scope.session < 0):
        print("ERROR_OSC Session didnt open \n")
        Stop_program = 1
    else: print("Oscilloscope session - OK\n")

  #Открываем сессию с генератором
    #status = viOpen(rmSession, usbGEN, VI_NULL, VI_NULL, gen)
    if (gen.session < 0):
        printf("ERROR_GEN Generator session didnt open\n")
        Stop_program = 1
    else: print("Generator session - OK\n")


  #НАСТРОЙКА ОСЦИЛЛОГРАФА

  #Останавливаем осцилограф для настройки (странно, но это надо сделать иначе запись потом не работает)
    scope.write(":STOP\n")
    #viWrite(scope, ":STOP\n", 512, VI_NULL)
    
    scope.write("status:measurement:enable 512; *sre 1")#мб фигня и надо не так, но не надо ремень, мы не разобрались с 512(((
  #Настройка записи отсчетов в память устройства(осцилограф)
    scope.write(":WAV:MODE NORMal\n")
    #viWrite(scope, ":WAV:MODE NORMal\n", 512, VI_NULL)

  #Настройка типа записываемых данных(осцилограф)
    scope.write(":WAV:FORM ASCII\n")
    #viWrite(scope, ":WAV:FORM ASCII\n", 512, VI_NULL)

  #Настройка количества хранимых данных(осцилограф)
    scope.write(":WAV:STOP 1200\n")
    #viWrite(scope, ":WAV:STOP 1200\n", 512, VI_NULL)

  #Настройка триггера по каналу EXT(должно сработать)(осцилограф)
    #viWrite(scope, (ViBuf)":TRIGger:EDGe:SOURce EXT\n", 512, VI_NULL)
    scope.write(":TRIGger:EDGe:SOURce EXT\n")

  #Настройка типа работы по триггеру одиночный мод *останавливается по записи 12000 отсчетов включится по следующему триггеру*(осцилограф)
  #viWrite(scope, (ViBuf)":TRIGger:SWEep SINGle\n", 512, VI_NULL)
  #viWrite(scope, (ViBuf)":TRIGger:SWEep NORMal\n", 512, VI_NULL)
    scope.write(":TRIGger:SWEep NORMal\n")

  #Запускаем осциллограф
    scope.write(":RUN\n")
    #viWrite(scope, ":RUN\n", 512, VI_NULL)


  #НАСТРОЙКА ГЕНЕРАТОРА
  #Выключаем канал генератора для настройки
    gen.write(":OUTP1 OFF\n")
    #viWrite(gen, ":OUTP1 OFF\n", 512, VI_NULL)
                
    gen.write("status:measurement:enable 512; *sre 1")
  #Настройка типа сигнала *синусоида, частота Гц, амплитуда В, задержка, начальная фаза*(генератор) 
    gen.write(":SOUR1:APPL:SIN 3200000,5,0,0\n")
    #viWrite(gen, ":SOUR1:APPL:SIN 3200000,5,0,0\n", 512, VI_NULL)
    gen.write(":SOUR1:BURSt ON\n")
    #viWrite(gen, ":SOUR1:BURSt ON\n", 512, VI_NULL)
#Настройка импульсного режима для первого канала *количество периодов синусоиды*(генератор)
    gen.write(":SOUR1:BURSt:NCYCles 10\n")
    #viWrite(gen, ":SOUR1:BURSt:NCYCles 10\n", 512, VI_NULL)

  #Настройка импульсного режима для первого канала (генератор)
    gen.write(":SOUR1:BURSt:INT:PERiod 0.01\n")
    #viWrite(gen, ":SOUR1:BURSt:INT:PERiod 0.01\n", 512, VI_NULL)

  #Настройка импульсного режима для первого канала *запуск вручную*(генератор)
  #viWrite(gen, (ViBuf)":SOUR1:BURSt:TRIG:SOUR MANual\n", 512, VI_NULL)

  #Настройка синхронного первого канала *Для выдачи синхросигнала осцилографу*(генератор)
    gen.write(":OUTP1:SYNC ON\n")
    #viWrite(gen, ":OUTP1:SYNC ON\n", 512, VI_NULL)

    Sleep(1000)

  #Включаем генератор
    gen.write(":OUTP1 ON\nn")
    #viWrite(gen, ":OUTP1 ON\n", 512, VI_NULL)
    gen.write(":OUTP1:TRIG:DEL 0.000000083\n")
    #viWrite(gen, ":OUTP1:TRIG:DEL 0.000000083\n", 512, VI_NULL) #сдвигаем триггер на 83 нс
    gen.write(":ACQuire:AVERages 128\n")         
    #viWrite(gen, ":ACQuire:AVERages 128\n", 512, VI_NULL) #усреднение по 8 осциллограммам
#функция освобождения приборов


In [21]:
def devices_out():
  #Закрываем сессии с устройствами
    viClose(scope)
    viClose(gen)
  #Закрываем сессию с ресурсным менеджером
    viClose(rmSession)

In [19]:
SettingUpDevices() #настраиваем приборы
devices_out() #отпускаем приборы
  


VisaIOError: VI_ERROR_INV_RSRC_NAME (-1073807342): Invalid resource reference specified. Parsing error.